# Highlight differences between hu.MAP 2.0 and hu.MAP 3.0 data

This notebook is primarily an intermediate to introducing to the next notebook in this series, ['Using snakemake to highlight differences between hu.MAP 2.0 and hu.MAP 3.0 data for multiple identifiers'](Using_snakemake_to_highlight_differences_between_hu.MAP2_and_hu.MAP3_data_for_multiple_indentifiers..ipynb). Here the steps to see the difference between complexes represented in hu.MAP 2.0 and hu.MAP 3.0 data is done with one example. You can change the hard-coded values here to investigate the differences for others. However, chances are you are interested in the differnces for several genes/proteins. In ['Using snakemake to highlight differences between hu.MAP 2.0 and hu.MAP 3.0 data for multiple identifiers'](Using_snakemake_to_highlight_differences_between_hu.MAP2_and_hu.MAP3_data_for_multiple_indentifiers.ipynb), it will work through the steps to run a Snakemake workflow to process the identifiers for many proteins/genes to generate summary reports highlighting differences in human complexes reported in hu.MAP 2.0 vs. hu.MAP 3.0 data. And that is more likely where you want to invest your time. You may want to quickly san the notebook below though to see the types of reports you'll expect when you give the snakemake file your list of identifiers. And with more details here it may help you troubleshoot such snakemake-generated reports. 

This effort just is meant to highlight possible differences. There is some approximating done to determine possible corresponding complexes that inherently brings in some possible judgement calls. You'll need to explore the results from each for yourself to compare in depth. The hu.MAP 2.0 data can be explored in sessions launched from my [humap2-binder repo](https://github.com/fomightez/humap2-binder).

------------

## Step #1: Preparation

The preparation parallels the notebooks ['Use of hu.MAP 3.0 data programmatically with Python, taking advantage of Jupyter features'](Working_with_hu.MAP3_data_with_Python_in_Jupyter_Basics.ipynb) and ['Use of hu.MAP 2.0 data programmatically with Python, taking advantage of Jupyter features'](Working_with_hu.MAP3_data_with_Python_in_Jupyter_Basics.ipynb), and so see them for more insight.

Get the data for hu.MAP 2.0 and hu.MAP 3.0:

In [1]:
!curl -OL https://raw.githubusercontent.com/fomightez/humap2-binder/refs/heads/main/additional_nbs/standardizing_initial_data/humap2_complexes_20200809InOrderMatched.csv
!curl -OL https://raw.githubusercontent.com/fomightez/humap3-binder/refs/heads/main/additional_nbs/standardizing_initial_data/hu.MAP3.0_complexes_wConfidenceScores_total15326_wGenenames_20240922InOrderMatched.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  502k  100  502k    0     0   990k      0 --:--:-- --:--:-- --:--:--  991k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1243k  100 1243k    0     0  2090k      0 --:--:-- --:--:-- --:--:-- 2090k


Get an accessory script for adding information about the proteins in the complexes:

In [2]:
!curl -OL https://raw.githubusercontent.com/fomightez/structurework/refs/heads/master/humap3-utilities/make_lookup_table_for_extra_info4complexes.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2893  100  2893    0     0   6999      0 --:--:-- --:--:-- --:--:--  6987


##### Put the data on the complexes into Pandas dataframe

In [3]:
!curl -OL https://raw.githubusercontent.com/fomightez/structurework/refs/heads/master/humap3-utilities/complexes_rawCSV_to_df.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1007  100  1007    0     0   3210      0 --:--:-- --:--:-- --:--:--  3217


In [4]:
!uv run complexes_rawCSV_to_df.py humap2_complexes_20200809InOrderMatched.csv
import pandas as pd
rd2_df = pd.read_pickle('raw_complexes_pickled_df.pkl')
rd2_df

Reading inline script metadata from `complexes_rawCSV_to_df.py`
Installed 10 packages in 163ms                                       


,HuMAP2_ID,Confidence,Uniprot_ACCs,genenames
0,HuMAP2_00000,3,O95900 Q9BQS8,TRUB2 FYCO1
1,HuMAP2_00001,4,Q15102 P68402 Q15797 P08133 Q99426 Q9H4M9,PAFAH1B3 PAFAH1B2 SMAD1 ANXA6 TBCB EHD1
2,HuMAP2_00002,5,Q9UF11 A1KXE4 Q6ZRY4 Q9Y6M7 Q15038 O43251 Q930...,PLEKHB1 FAM168B RBPMS2 SLC4A7 DAZAP2 RBFOX2 RB...
3,HuMAP2_00003,5,O14974 Q8WUM9 Q9Y5Y0 Q16563 Q14919 Q15836 Q299...,PPP1R12A SLC20A1 FLVCR1 SYPL1 DRAP1 VAMP3 MICA...
4,HuMAP2_00004,4,Q8WV99 Q49A92 Q9NQT8 Q9H672 P20774,ZFAND2B C8orf34 KIF13B ASB7 OGN
...,...,...,...,...
6960,HuMAP2_07014,4,Q9HC97 P31152 Q6S8J3 P13727 Q92871,GPR35 MAPK4 POTEE PRG2 PMM1
6961,HuMAP2_07015,4,Q96E29 Q8N5N7 Q96I51 Q9H5L6 O75127 Q9NPE2,MTERF3 MRPL50 RCC1L THAP9 PTCD1 NGRN
6962,HuMAP2_07016,5,O75319 Q96HN2 Q8NE31 O43865 P52952 Q2T9J0 Q9UP...,DUSP11 AHCYL2 FAM13C AHCYL1 NKX2-5 TYSND1 PDZR...
6963,HuMAP2_07017,2,Q96GP6 Q53GT1 P49448,SCARF2 KLHL22 GLUD2


In [5]:
!uv run complexes_rawCSV_to_df.py hu.MAP3.0_complexes_wConfidenceScores_total15326_wGenenames_20240922InOrderMatched.csv
import pandas as pd
rd3_df = pd.read_pickle('raw_complexes_pickled_df.pkl')
rd3_df

Reading inline script metadata from `complexes_rawCSV_to_df.py`
⠙                                                                               

,HuMAP3_ID,ComplexConfidence,Uniprot_ACCs,genenames
0,huMAP3_00000.1,1,Q9UGQ2 P20963 Q9NWV4,CACFD1 CD247 CZIB
1,huMAP3_00001.1,1,Q9NWB1 O94887 Q9NQ92,RBFOX1 FARP2 COPRS
2,huMAP3_00002.1,1,Q8N3D4 Q9Y3A4,EHBP1L1 RRP7A
3,huMAP3_00003.1,1,Q5T2D2 O00429,TREML2 DNM1L
4,huMAP3_00004.1,1,Q9H9C1 Q9H267 O95460 P21941 P78540,VIPAS39 VPS33B MATN4 MATN1 ARG2
...,...,...,...,...
15321,huMAP3_15345.1,6,O14628 Q3SXZ3,ZNF195 ZNF718
15322,huMAP3_15346.1,6,Q6ZWT7 P08910 Q86VD1 Q9UJQ1 Q9Y6X9,MBOAT2 ABHD2 MORC1 LAMP5 MORC2
15323,huMAP3_15347.1,6,A6ND91 Q4V339,ASPDH ZNG1F
15324,huMAP3_15348.1,6,A6NKF2 P08217 Q8IVW6 Q99856,ARID3C CELA2A ARID3B ARID3A




--------

## Analyze complexes in hu.MAP 2 vs. hu.MAP3 for a protein


In [6]:
search_term = "ROGDI"

Get complexes that protein is in in both sets of data, adding extra information like done in earlier notebooks.

In [7]:
# Next few cells will run the query collecting all complexes it occurs with and adding details for both datasets
pattern = fr'\b{search_term}\b' # Create a regex pattern with word boundaries
d2_rows_with_term_df = rd2_df[rd2_df['Uniprot_ACCs'].str.contains(pattern, case=False, regex=True) | rd2_df['genenames'].str.contains(pattern, case=False, regex=True)].copy()
d3_rows_with_term_df = rd3_df[rd3_df['Uniprot_ACCs'].str.contains(pattern, case=False, regex=True) | rd3_df['genenames'].str.contains(pattern, case=False, regex=True)].copy()
# make the dataframe have each row be a single protein
# to prepare to use pandas `explode()` to do that, first make the content in be lists
d2_rows_with_term_df['Uniprot_ACCs'] = d2_rows_with_term_df['Uniprot_ACCs'].str.split()
d3_rows_with_term_df['Uniprot_ACCs'] = d3_rows_with_term_df['Uniprot_ACCs'].str.split()
d2_rows_with_term_df['genenames'] = d2_rows_with_term_df['genenames'].str.split()
d3_rows_with_term_df['genenames'] = d3_rows_with_term_df['genenames'].str.split()
# Now use explode to create a new row for each element in both columns
df2_expanded = d2_rows_with_term_df.explode(['Uniprot_ACCs', 'genenames']).copy()
df3_expanded = d3_rows_with_term_df.explode(['Uniprot_ACCs', 'genenames']).copy()
# Reset the index 
df2_expanded = df2_expanded.reset_index(drop=True)
df3_expanded = df3_expanded.reset_index(drop=True)
# Display the first few rows of the expanded dataframe
print(df2_expanded.tail())
print(" ")
print(df3_expanded.tail())
# Next add extra information from UniProt for each protein

       HuMAP2_ID  Confidence Uniprot_ACCs genenames
6   HuMAP2_01834           3       Q9Y4E6      WDR7
7   HuMAP2_03388           2       Q9Y485     DMXL1
8   HuMAP2_03388           2       Q9GZN7     ROGDI
9   HuMAP2_03388           2       Q8TDJ6     DMXL2
10  HuMAP2_03388           2       Q9Y4E6      WDR7
 
          HuMAP3_ID  ComplexConfidence Uniprot_ACCs genenames
297  huMAP3_13872.1                  6       Q8NI08     NCOA7
298  huMAP3_13872.1                  6       Q8TDJ6     DMXL2
299  huMAP3_13872.1                  6       Q9GZN7     ROGDI
300  huMAP3_13872.1                  6       Q9Y485     DMXL1
301  huMAP3_13872.1                  6       Q9Y4E6      WDR7


In [8]:
# This cell makes lookup table with the extra information; it takes a while to run & so is in a cell on its own to save time during development
df_expanded = pd.concat([df2_expanded,df3_expanded])
%run -i make_lookup_table_for_extra_info4complexes.py

In [9]:
# Use collected information to enhance the dataframes
pn_dict = {k: v['protein_name'] for k, v in lookup_dict.items()}
disease_dict = {k: v['disease'] for k, v in lookup_dict.items()}
synonyms_dict = {k: v['synonyms'] for k, v in lookup_dict.items()}
df3_expanded['synonyms'] = df3_expanded['Uniprot_ACCs'].map(synonyms_dict)
df3_expanded['protein_name'] = df3_expanded['Uniprot_ACCs'].map(pn_dict)
df3_expanded['disease'] = df3_expanded['Uniprot_ACCs'].map(disease_dict)
conf_val2text_dict = {
    1: 'Extremely High',
    2: 'Very High',
    3: 'High',
    4: 'Moderate High',
    5: 'Medium High',
    6: 'Medium'
}
# Use vectorized mapping to convert confidence values to text
df3_expanded['ComplexConfidence'] = df3_expanded['ComplexConfidence'].map(conf_val2text_dict)
base_uniprot_url = 'https://www.uniprot.org/uniprotkb/'
df3_expanded = df3_expanded.assign(Link=base_uniprot_url + df3_expanded['Uniprot_ACCs'])

# do same for 2.0 data
df2_expanded['synonyms'] = df2_expanded['Uniprot_ACCs'].map(synonyms_dict)
df2_expanded['protein_name'] = df2_expanded['Uniprot_ACCs'].map(pn_dict)
df2_expanded['disease'] = df2_expanded['Uniprot_ACCs'].map(disease_dict)
conf_val2text_dict = {
    1: 'Extremely High',
    2: 'Very High',
    3: 'High',
    4: 'Moderate High',
    5: 'Medium High',
    6: 'Medium'
}
# Use vectorized mapping to convert confidence values to text
df2_expanded['Confidence'] = df2_expanded['Confidence'].map(conf_val2text_dict)
base_uniprot_url = 'https://www.uniprot.org/uniprotkb/'
df2_expanded = df2_expanded.assign(Link=base_uniprot_url + df2_expanded['Uniprot_ACCs'])
# Display the first few rows of the expanded dataframe
#display(df2_expanded.tail())
#display(df3_expanded.tail())

Saving that as tab-separated data.

In [10]:
import datetime
now = datetime.datetime.now()
df2_expanded.to_csv(f'{search_term}_complexesHUMAP2{now.strftime("_%Y_%m_%d")}.tsv', sep='\t',index = False) 
df3_expanded.to_csv(f'{search_term}_complexesHUMAP3{now.strftime("_%Y_%m_%d")}.tsv', sep='\t',index = False)

In [11]:
ls *_complexesHUMAP2*

ROGDI_complexesHUMAP2_2024_12_02.tsv


In [12]:
ls *_complexesHUMAP3*

ROGDI_complexesHUMAP3_2024_12_02.tsv


In [13]:
print(df2_expanded.tail())
print(df3_expanded.tail())

       HuMAP2_ID Confidence Uniprot_ACCs genenames        synonyms  \
6   HuMAP2_01834       High       Q9Y4E6      WDR7  KIAA0541; TRAG   
7   HuMAP2_03388  Very High       Q9Y485     DMXL1             XL1   
8   HuMAP2_03388  Very High       Q9GZN7     ROGDI   None reported   
9   HuMAP2_03388  Very High       Q8TDJ6     DMXL2        KIAA0856   
10  HuMAP2_03388  Very High       Q9Y4E6      WDR7  KIAA0541; TRAG   

                      protein_name  \
6   WD repeat-containing protein 7   
7               DmX-like protein 1   
8            Protein rogdi homolog   
9               DmX-like protein 2   
10  WD repeat-containing protein 7   

                                              disease  \
6                                       None reported   
7                                       None reported   
8                        Kohlschuetter-Toenz syndrome   
9   Polyendocrine-polyneuropathy syndrome; Deafnes...   
10                                      None reported   

       

In [14]:
# Get the size of each group in df2_expanded
df2_group_sizes = df2_expanded.groupby('HuMAP2_ID').size()
df2_max_size = df2_group_sizes.max()

# Get the size of each group in df3_expanded
df3_group_sizes = df3_expanded.groupby('HuMAP3_ID').size()
df3_max_size = df3_group_sizes.max()

# Print the results
print(f"Maximum group size in df2_expanded (HuMAP2_ID): {df2_max_size}")
print(f"Maximum group size in df3_expanded (HuMAP3_ID): {df3_max_size}")

# Determine which has the larger maximum group size
if df2_max_size > df3_max_size:
    print(f"df2_expanded has the larger maximum group size ({df2_max_size})")
elif df3_max_size > df2_max_size:
    print(f"df3_expanded has the larger maximum group size ({df3_max_size})")
else:
    print("Both dataframes have the same maximum group size")

# Optionally, to see which groups have these maximum sizes:
print("\nGroups with maximum size in df2_expanded:")
print(df2_group_sizes[df2_group_sizes == df2_max_size])
print("\nGroups with maximum size in df3_expanded:")
print(df3_group_sizes[df3_group_sizes == df3_max_size])

Maximum group size in df2_expanded (HuMAP2_ID): 5
Maximum group size in df3_expanded (HuMAP3_ID): 56
df3_expanded has the larger maximum group size (56)

Groups with maximum size in df2_expanded:
HuMAP2_ID
HuMAP2_01834    5
dtype: int64

Groups with maximum size in df3_expanded:
HuMAP3_ID
huMAP3_12042.1    56
dtype: int64


In [15]:
df2_group_sizes = df2_expanded.groupby('HuMAP2_ID').size()
df2_group_sizes

HuMAP2_ID
HuMAP2_01148    2
HuMAP2_01834    5
HuMAP2_03388    4
dtype: int64

In [16]:
df3_group_sizes = df3_expanded.groupby('HuMAP3_ID').size()
df3_group_sizes

HuMAP3_ID
huMAP3_01501.1    15
huMAP3_01899.1    11
huMAP3_03469.1     4
huMAP3_05952.1     2
huMAP3_07099.1    18
huMAP3_07329.1    20
huMAP3_08381.1    20
huMAP3_08678.1    22
huMAP3_09242.1    20
huMAP3_09477.1    26
huMAP3_09873.1    24
huMAP3_10511.1    35
huMAP3_12042.1    56
huMAP3_13872.1    29
dtype: int64

In [17]:
# Sort groups by size for both dataframes
df2_sorted = df2_expanded.groupby('HuMAP2_ID').size().sort_values(ascending=False)
df3_sorted = df3_expanded.groupby('HuMAP3_ID').size().sort_values(ascending=False)

# Print the sorted group sizes
print("Sorted HuMAP2_ID group sizes:")
print(df2_sorted)
print("\nSorted HuMAP3_ID group sizes:")
print(df3_sorted)

# Create a DataFrame to show the correspondences
# First, convert the series to dataframes
df2_ranks = df2_sorted.reset_index()
df3_ranks = df3_sorted.reset_index()

# Rename columns for clarity
df2_ranks.columns = ['HuMAP2_ID', 'size_df2']
df3_ranks.columns = ['HuMAP3_ID', 'size_df3']

# Add rank columns
df2_ranks['rank'] = range(1, len(df2_ranks) + 1)
df3_ranks['rank'] = range(1, len(df3_ranks) + 1)

# Merge the rankings side by side
correspondence = pd.merge(
    df2_ranks,
    df3_ranks,
    on='rank',
    how='outer'
)

# Sort by rank for clear viewing
correspondence = correspondence.sort_values('rank')

print("\nCorrespondence by size rank:")
print(correspondence)

# To verify if the sizes match exactly
print("\nDo all sizes match?")
print((correspondence['size_df2'] == correspondence['size_df3']).all())

Sorted HuMAP2_ID group sizes:
HuMAP2_ID
HuMAP2_01834    5
HuMAP2_03388    4
HuMAP2_01148    2
dtype: int64

Sorted HuMAP3_ID group sizes:
HuMAP3_ID
huMAP3_12042.1    56
huMAP3_10511.1    35
huMAP3_13872.1    29
huMAP3_09477.1    26
huMAP3_09873.1    24
huMAP3_08678.1    22
huMAP3_09242.1    20
huMAP3_07329.1    20
huMAP3_08381.1    20
huMAP3_07099.1    18
huMAP3_01501.1    15
huMAP3_01899.1    11
huMAP3_03469.1     4
huMAP3_05952.1     2
dtype: int64

Correspondence by size rank:
       HuMAP2_ID  size_df2  rank       HuMAP3_ID  size_df3
0   HuMAP2_01834       5.0     1  huMAP3_12042.1        56
1   HuMAP2_03388       4.0     2  huMAP3_10511.1        35
2   HuMAP2_01148       2.0     3  huMAP3_13872.1        29
3            NaN       NaN     4  huMAP3_09477.1        26
4            NaN       NaN     5  huMAP3_09873.1        24
5            NaN       NaN     6  huMAP3_08678.1        22
6            NaN       NaN     7  huMAP3_09242.1        20
7            NaN       NaN     8  huMAP3_07

In [18]:
# Show only top N correspondences (e.g., top 10)
N = 10
print(f"\nTop {N} correspondences:")
print(correspondence.head(N))


Top 10 correspondences:
      HuMAP2_ID  size_df2  rank       HuMAP3_ID  size_df3
0  HuMAP2_01834       5.0     1  huMAP3_12042.1        56
1  HuMAP2_03388       4.0     2  huMAP3_10511.1        35
2  HuMAP2_01148       2.0     3  huMAP3_13872.1        29
3           NaN       NaN     4  huMAP3_09477.1        26
4           NaN       NaN     5  huMAP3_09873.1        24
5           NaN       NaN     6  huMAP3_08678.1        22
6           NaN       NaN     7  huMAP3_09242.1        20
7           NaN       NaN     8  huMAP3_07329.1        20
8           NaN       NaN     9  huMAP3_08381.1        20
9           NaN       NaN    10  huMAP3_07099.1        18


In [19]:
# Cell above close but this looks better without the `3           NaN       NaN     `-like parts
# Sort by rank
correspondence = correspondence.sort_values('rank')

# Convert sizes to integers
correspondence['size_df2'] = correspondence['size_df2'].fillna(0).astype(int)
correspondence['size_df3'] = correspondence['size_df3'].fillna(0).astype(int)

# Create custom string representation
print("\nCorrespondence by size rank:")
print("       HuMAP2_ID  size_df2  rank       HuMAP3_ID  size_df3")
for _, row in correspondence.iterrows():
    humap2_str = str(row['HuMAP2_ID']).ljust(12) if not pd.isna(row['HuMAP2_ID']) else " " * 12
    size2_str = str(row['size_df2']).rjust(8) if row['size_df2'] != 0 else " " * 8
    rank_str = str(row['rank']).rjust(6)
    humap3_str = str(row['HuMAP3_ID']).ljust(15)
    size3_str = str(row['size_df3']).rjust(8)
    
    if pd.isna(row['HuMAP2_ID']):
        print(f"{' ' * 20}{rank_str}  {humap3_str}{size3_str}")
    else:
        print(f"{humap2_str}{size2_str}{rank_str}  {humap3_str}{size3_str}")


Correspondence by size rank:
       HuMAP2_ID  size_df2  rank       HuMAP3_ID  size_df3
HuMAP2_01834       5     1  huMAP3_12042.1       56
HuMAP2_03388       4     2  huMAP3_10511.1       35
HuMAP2_01148       2     3  huMAP3_13872.1       29
                         4  huMAP3_09477.1       26
                         5  huMAP3_09873.1       24
                         6  huMAP3_08678.1       22
                         7  huMAP3_09242.1       20
                         8  huMAP3_07329.1       20
                         9  huMAP3_08381.1       20
                        10  huMAP3_07099.1       18
                        11  huMAP3_01501.1       15
                        12  huMAP3_01899.1       11
                        13  huMAP3_03469.1        4
                        14  huMAP3_05952.1        2


In [20]:
# Alternative to cell just above using Pandas' built-in display options
# Convert sizes to integers
# Convert sizes to integers
correspondence['size_df2'] = correspondence['size_df2'].fillna(0).astype(int)
correspondence['size_df3'] = correspondence['size_df3'].fillna(0).astype(int)

# Replace NaN values with empty strings for display
display_df = correspondence.copy()
display_df['HuMAP2_ID'] = display_df['HuMAP2_ID'].fillna('')
display_df['size_df2'] = display_df['size_df2'].replace(0, '')

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("\nCorrespondence by size rank:")
print(display_df)


Correspondence by size rank:
       HuMAP2_ID size_df2  rank       HuMAP3_ID  size_df3
0   HuMAP2_01834        5     1  huMAP3_12042.1        56
1   HuMAP2_03388        4     2  huMAP3_10511.1        35
2   HuMAP2_01148        2     3  huMAP3_13872.1        29
3                             4  huMAP3_09477.1        26
4                             5  huMAP3_09873.1        24
5                             6  huMAP3_08678.1        22
6                             7  huMAP3_09242.1        20
7                             8  huMAP3_07329.1        20
8                             9  huMAP3_08381.1        20
9                            10  huMAP3_07099.1        18
10                           11  huMAP3_01501.1        15
11                           12  huMAP3_01899.1        11
12                           13  huMAP3_03469.1         4
13                           14  huMAP3_05952.1         2


In [21]:
# Even cleaner Alternative to cell just above using Pandas' built-in display options
# Convert sizes to integers
correspondence['size_df2'] = correspondence['size_df2'].fillna(0).astype(int)
correspondence['size_df3'] = correspondence['size_df3'].fillna(0).astype(int)

# Replace NaN values with empty strings for display
display_df = correspondence.copy()
display_df['HuMAP2_ID'] = display_df['HuMAP2_ID'].fillna('')
display_df['size_df2'] = display_df['size_df2'].replace(0, '')

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("\nCorrespondence by size rank:")
print(display_df.to_string(index=False))


Correspondence by size rank:
   HuMAP2_ID size_df2  rank      HuMAP3_ID  size_df3
HuMAP2_01834        5     1 huMAP3_12042.1        56
HuMAP2_03388        4     2 huMAP3_10511.1        35
HuMAP2_01148        2     3 huMAP3_13872.1        29
                          4 huMAP3_09477.1        26
                          5 huMAP3_09873.1        24
                          6 huMAP3_08678.1        22
                          7 huMAP3_09242.1        20
                          8 huMAP3_07329.1        20
                          9 huMAP3_08381.1        20
                         10 huMAP3_07099.1        18
                         11 huMAP3_01501.1        15
                         12 huMAP3_01899.1        11
                         13 huMAP3_03469.1         4
                         14 huMAP3_05952.1         2


In [22]:
# USe 'even cleaner Alternative' to do what cell below does: # Add a column showing the size difference
# Convert sizes to integers
# Sort groups by size for both dataframes
df2_sorted = df2_expanded.groupby('HuMAP2_ID').size().sort_values(ascending=False)
df3_sorted = df3_expanded.groupby('HuMAP3_ID').size().sort_values(ascending=False)


# Create a DataFrame to show the correspondences
# First, convert the series to dataframes
df2_ranks = df2_sorted.reset_index()
df3_ranks = df3_sorted.reset_index()

# Rename columns for clarity
df2_ranks.columns = ['HuMAP2_ID', 'size_df2']
df3_ranks.columns = ['HuMAP3_ID', 'size_df3']

# Add rank columns
df2_ranks['rank'] = range(1, len(df2_ranks) + 1)
df3_ranks['rank'] = range(1, len(df3_ranks) + 1)

# Merge the rankings side by side
size_rank_df = pd.merge(
    df2_ranks,
    df3_ranks,
    on='rank',
    how='outer'
)

# Sort by rank for clear viewing
size_rank_df = size_rank_df.sort_values('rank')
# Convert sizes to integers
size_rank_df['size_df2'] = size_rank_df['size_df2'].fillna(0).astype(int)
size_rank_df['size_df3'] = size_rank_df['size_df3'].fillna(0).astype(int)
#size_rank_df['improvement_in_v3'] =  size_rank_df['size_df3'] - size_rank_df['size_df2']
size_rank_df['improvement_in_v3'] = size_rank_df.apply(
    lambda row: row['size_df3'] - row['size_df2'] if not pd.isna(row['HuMAP2_ID']) else 'na', 
    axis=1
)

# Replace NaN values with empty strings for display
display_df = size_rank_df.copy()
display_df['HuMAP2_ID'] = display_df['HuMAP2_ID'].fillna('')
display_df['size_df2'] = display_df['size_df2'].replace(0, '')

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("\nSize rank for version 2 vs. 3 with 'improvement' in version 3 indicated:")
print(display_df.to_string(index=False))


Size rank for version 2 vs. 3 with 'improvement' in version 3 indicated:
   HuMAP2_ID size_df2  rank      HuMAP3_ID  size_df3 improvement_in_v3
HuMAP2_01834        5     1 huMAP3_12042.1        56                51
HuMAP2_03388        4     2 huMAP3_10511.1        35                31
HuMAP2_01148        2     3 huMAP3_13872.1        29                27
                          4 huMAP3_09477.1        26                na
                          5 huMAP3_09873.1        24                na
                          6 huMAP3_08678.1        22                na
                          7 huMAP3_09242.1        20                na
                          8 huMAP3_07329.1        20                na
                          9 huMAP3_08381.1        20                na
                         10 huMAP3_07099.1        18                na
                         11 huMAP3_01501.1        15                na
                         12 huMAP3_01899.1        11                na
   

In [23]:
## Add number of shared items
# First, get unique HuMAP3_IDs
humap3_ids = df3_expanded['HuMAP3_ID'].unique()
shared_items_dict = {}

df2_groups = df2_expanded.groupby('HuMAP2_ID')['Uniprot_ACCs'].apply(list)
df3_groups = df3_expanded.groupby('HuMAP3_ID')['Uniprot_ACCs'].apply(list)

# Convert lists to sets
df2_sets = {idx: set(ids) for idx, ids in df2_groups.items()}
df3_sets = {idx: set(ids) for idx, ids in df3_groups.items()}

# Function to check shared items
def get_shared_items(set1, set2):
    return set1.intersection(set2)

# Populate the nested dictionary from the dataframes - keys for the `shared_items_dict` will be the 'HuMAP2_ID' ids with the values for each being a dictionary with the 'HuMAP3_ID' as the keys with the value of number of shared items
for _, row in df2_expanded.iterrows():
    humap2_id = row['HuMAP2_ID']
    
    # Initialize nested dictionary if HuMAP2_ID doesn't exist
    if humap2_id not in shared_items_dict:
        shared_items_dict[humap2_id] = {}
        # For each unique HuMAP3_ID, find shared items
        for humap3_id in humap3_ids:
            shared_ids = get_shared_items(df2_sets[humap2_id], df3_sets[humap3_id])
            shared_items_dict[humap2_id][humap3_id] = len(shared_ids)

'''# Debugging function to print out the dictionary
def print_shared_items_debug(humap2_id):
    print(f"\nDebugging shared items for {humap2_id}:")
    if humap2_id in shared_items_dict:
        for humap3_id in humap3_ids:
            print(f"  {humap3_id}: {shared_items_dict[humap2_id][humap3_id]} shared Uniprot_ACCs")
    else:
        print("  No entries found")
'''




# Add a new column for shared items to the correspondence DataFrame
#size_rank_df['shared_items'] = size_rank_df['HuMAP2_ID'].map(shared_items_dict).fillna('na')
#size_rank_df['shared_items'] = size_rank_df['shared_items'].apply(lambda x: int(x) if isinstance(x, (int, float)) and not pd.isna(x) else 'na')
#size_rank_df['shared_items'] = size_rank_df['HuMAP2_ID'].map(shared_items_dict).apply(lambda x: int(x) if isinstance(x, (int, float)) and not pd.isna(x) else 'na')
def get_shared_items(ids):
    humap2_id = ids.iloc[0]
    humap3_id = ids.iloc[1]
    if humap2_id in shared_items_dict:
        return shared_items_dict[humap2_id][humap3_id]
    return 'na'

# Optional: Add debugging print statement for the first few rows
#print_shared_items_debug('HuMAP2_01834')

# Apply the new method to get shared items
size_rank_df['shared_items'] = size_rank_df[['HuMAP2_ID','HuMAP3_ID']].apply(get_shared_items, axis=1)

# When preparing for display, modify the display_df creation
display_df = size_rank_df.copy()
display_df['HuMAP2_ID'] = display_df['HuMAP2_ID'].fillna('')
display_df['size_df2'] = display_df['size_df2'].replace(0, '')
display_df['shared_items'] = display_df['shared_items'].replace('', 'na')

# Update print statement
print("\nSize rank for version 2 vs. 3 with 'improvement' in version 3 and shared items:")
print(display_df.to_string(index=False))


Size rank for version 2 vs. 3 with 'improvement' in version 3 and shared items:
   HuMAP2_ID size_df2  rank      HuMAP3_ID  size_df3 improvement_in_v3 shared_items
HuMAP2_01834        5     1 huMAP3_12042.1        56                51            4
HuMAP2_03388        4     2 huMAP3_10511.1        35                31            3
HuMAP2_01148        2     3 huMAP3_13872.1        29                27            2
                          4 huMAP3_09477.1        26                na           na
                          5 huMAP3_09873.1        24                na           na
                          6 huMAP3_08678.1        22                na           na
                          7 huMAP3_09242.1        20                na           na
                          8 huMAP3_07329.1        20                na           na
                          9 huMAP3_08381.1        20                na           na
                         10 huMAP3_07099.1        18                na         

In [24]:
# Add a new column for coverage ratio
size_rank_df['coverage_ratio'] = size_rank_df.apply(
    lambda row: round(row['shared_items'] / min(row['size_df2'], row['size_df3']) * 100, 2) 
    if row['shared_items'] != 'na' and row['size_df2'] > 0 and row['size_df3'] > 0 
    else 'na', 
    axis=1
)

# Modify display preparation
display_df = size_rank_df.copy()
display_df['HuMAP2_ID'] = display_df['HuMAP2_ID'].fillna('')
display_df['size_df2'] = display_df['size_df2'].replace(0, '')
display_df['shared_items'] = display_df['shared_items'].replace('', 'na')

# Update print statement
print("\nSize rank for version 2 vs. 3 with improvement, shared items, and coverage:")
print(display_df.to_string(index=False))


Size rank for version 2 vs. 3 with improvement, shared items, and coverage:
   HuMAP2_ID size_df2  rank      HuMAP3_ID  size_df3 improvement_in_v3 shared_items coverage_ratio
HuMAP2_01834        5     1 huMAP3_12042.1        56                51            4           80.0
HuMAP2_03388        4     2 huMAP3_10511.1        35                31            3           75.0
HuMAP2_01148        2     3 huMAP3_13872.1        29                27            2          100.0
                          4 huMAP3_09477.1        26                na           na             na
                          5 huMAP3_09873.1        24                na           na             na
                          6 huMAP3_08678.1        22                na           na             na
                          7 huMAP3_09242.1        20                na           na             na
                          8 huMAP3_07329.1        20                na           na             na
                          9 huMA

In [25]:
# Merge the rankings side by side in simple output
ssize_rank_df = pd.merge(
    df2_ranks,
    df3_ranks,
    on='rank',
    how='outer'
)

# Sort by rank and drop rows where either side is completely NaN
ssize_rank_df = ssize_rank_df.sort_values('rank')
ssize_rank_df_clean = ssize_rank_df.dropna(subset=['HuMAP2_ID', 'HuMAP3_ID'], how='any')

print("\nCorrespondence by size rank (without NaN rows):")
print(ssize_rank_df_clean)

# Get the correspondences as a dictionary or series
# This will match IDs based on their size rankings
size_corr_dict = dict(zip(ssize_rank_df_clean['HuMAP2_ID'], ssize_rank_df_clean['HuMAP3_ID']))

print("\nSize Correspondences (HuMAP2_ID -> HuMAP3_ID):")
for humap2, humap3 in size_corr_dict.items():
    print(f"{humap2} corresponds to {humap3} (sizes: {int(ssize_rank_df_clean[ssize_rank_df_clean['HuMAP2_ID'] == humap2]['size_df2'].iloc[0])} and {int(ssize_rank_df_clean[ssize_rank_df_clean['HuMAP3_ID'] == humap3]['size_df3'].iloc[0])})")


Correspondence by size rank (without NaN rows):
      HuMAP2_ID  size_df2  rank       HuMAP3_ID  size_df3
0  HuMAP2_01834       5.0     1  huMAP3_12042.1        56
1  HuMAP2_03388       4.0     2  huMAP3_10511.1        35
2  HuMAP2_01148       2.0     3  huMAP3_13872.1        29

Size Correspondences (HuMAP2_ID -> HuMAP3_ID):
HuMAP2_01834 corresponds to huMAP3_12042.1 (sizes: 5 and 56)
HuMAP2_03388 corresponds to huMAP3_10511.1 (sizes: 4 and 35)
HuMAP2_01148 corresponds to huMAP3_13872.1 (sizes: 2 and 29)


In [26]:
#Comparing the complex members to better see what corresponds

# First get the groups from both dataframes
df2_groups = df2_expanded.groupby('HuMAP2_ID')['Uniprot_ACCs'].apply(list)
df3_groups = df3_expanded.groupby('HuMAP3_ID')['Uniprot_ACCs'].apply(list)

# Convert lists to sets
source1_sets = {idx: set(ids) for idx, ids in df2_groups.items()}
source2_sets = {idx: set(ids) for idx, ids in df3_groups.items()}

def weighted_similarity(set1, set2, weight_jaccard=0.7):
    """Combined metric that considers both Jaccard similarity and raw intersection size"""
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    jaccard = len(intersection) / len(union) if union else 0
    max_possible_intersection = max(len(set1), len(set2))
    normalized_intersection = len(intersection) / max_possible_intersection if max_possible_intersection != 0 else 0
    
    return weight_jaccard * jaccard + (1 - weight_jaccard) * normalized_intersection

# Calculate similarities for all pairs
similarities = []
for humap2_id, set1 in source1_sets.items():
    for humap3_id, set2 in source2_sets.items():
        intersection = set1.intersection(set2)
        if len(intersection) >= 2:  # Minimum overlap constraint
            similarities.append({
                'HuMAP2_ID': humap2_id,
                'HuMAP3_ID': humap3_id,
                'weighted_similarity': weighted_similarity(set1, set2),
                'shared_items': len(intersection),
                'total_humap2': len(set1),
                'total_humap3': len(set2),
                'shared_elements': sorted(list(intersection))
            })

# Convert to DataFrame and sort by similarity
df_similarities = pd.DataFrame(similarities)
df_similarities = df_similarities.sort_values(['weighted_similarity', 'shared_items'], 
                                           ascending=[False, False])

# Print results
print("Top matches (with at least 2 shared items):")
print("\nDetailed view:")
for _, row in df_similarities.iterrows():
    print(f"\nHuMAP2 {row['HuMAP2_ID']} <-> HuMAP3 {row['HuMAP3_ID']}")
    print(f"Weighted similarity: {row['weighted_similarity']:.3f}")
    print(f"Shared items: {row['shared_items']} (out of {row['total_humap2']} in HuMAP2 and {row['total_humap3']} in HuMAP3)")
    print(f"Shared UniProt IDs: {', '.join(row['shared_elements'])}")

# Get best matches for each HuMAP2_ID
best_matches = df_similarities.loc[df_similarities.groupby('HuMAP2_ID')['weighted_similarity'].idxmax()]
print("\nBest match for each HuMAP2_ID:")
print(best_matches[['HuMAP2_ID', 'HuMAP3_ID', 'weighted_similarity', 'shared_items']].to_string())

# Get matches that share all members
perfect_subset_matches = df_similarities[
    df_similarities.apply(lambda x: 
        x['shared_items'] == min(x['total_humap2'], x['total_humap3']), 
    axis=1)
]

if not perfect_subset_matches.empty:
    print("\nPerfect subset matches (where all members of the smaller set are contained in the larger set):")
    print(perfect_subset_matches[['HuMAP2_ID', 'HuMAP3_ID', 'shared_items', 
                                'total_humap2', 'total_humap3']].to_string())

Top matches (with at least 2 shared items):

Detailed view:

HuMAP2 HuMAP2_03388 <-> HuMAP3 huMAP3_03469.1
Weighted similarity: 1.000
Shared items: 4 (out of 4 in HuMAP2 and 4 in HuMAP3)
Shared UniProt IDs: Q8TDJ6, Q9GZN7, Q9Y485, Q9Y4E6

HuMAP2 HuMAP2_01148 <-> HuMAP3 huMAP3_05952.1
Weighted similarity: 1.000
Shared items: 2 (out of 2 in HuMAP2 and 2 in HuMAP3)
Shared UniProt IDs: Q8TDJ6, Q9GZN7

HuMAP2 HuMAP2_01834 <-> HuMAP3 huMAP3_03469.1
Weighted similarity: 0.800
Shared items: 4 (out of 5 in HuMAP2 and 4 in HuMAP3)
Shared UniProt IDs: Q8TDJ6, Q9GZN7, Q9Y485, Q9Y4E6

HuMAP2 HuMAP2_01148 <-> HuMAP3 huMAP3_03469.1
Weighted similarity: 0.500
Shared items: 2 (out of 2 in HuMAP2 and 4 in HuMAP3)
Shared UniProt IDs: Q8TDJ6, Q9GZN7

HuMAP2 HuMAP2_03388 <-> HuMAP3 huMAP3_05952.1
Weighted similarity: 0.500
Shared items: 2 (out of 4 in HuMAP2 and 2 in HuMAP3)
Shared UniProt IDs: Q8TDJ6, Q9GZN7

HuMAP2 HuMAP2_01834 <-> HuMAP3 huMAP3_05952.1
Weighted similarity: 0.400
Shared items: 2 (out of 

In [27]:
# Try combining, because ranked by size probably correct if the number of shared items doesn't get any better if you re-order. Then fall back to ranking by share members
# if doesn't seem to work that the shared items never gets any better from ranked size comparisons.
# Get the groups from both dataframes
df2_groups = df2_expanded.groupby('HuMAP2_ID')['Uniprot_ACCs'].apply(list)
df3_groups = df3_expanded.groupby('HuMAP3_ID')['Uniprot_ACCs'].apply(list)

# Convert lists to sets
source1_sets = {idx: set(ids) for idx, ids in df2_groups.items()}
source2_sets = {idx: set(ids) for idx, ids in df3_groups.items()}

# Sort groups by size
source1_sorted = sorted(source1_sets.items(), key=lambda x: len(x[1]), reverse=True)
source2_sorted = sorted(source2_sets.items(), key=lambda x: len(x[1]), reverse=True)

# Function to check shared items
def get_shared_items(set1, set2):
    return set1.intersection(set2)

# Function for weighted similarity
def weighted_similarity(set1, set2, weight_jaccard=0.7):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    jaccard = len(intersection) / len(union) if union else 0
    max_possible_intersection = max(len(set1), len(set2))
    normalized_intersection = len(intersection) / max_possible_intersection if max_possible_intersection != 0 else 0
    
    return weight_jaccard * jaccard + (1 - weight_jaccard) * normalized_intersection

# Correspondence tracking
correspondences = []
used_source2_indices = set()

# First pass: Try to match by ranking and shared items
for i, (humap2_id, set1) in enumerate(source1_sorted):
    # If we've reached the end of source2 sorted list, break
    if i >= len(source2_sorted):
        break
    
    humap3_id, set2 = source2_sorted[i]
    
    # Check shared items
    shared = get_shared_items(set1, set2)
    
    if len(shared) > 1:  # need greater than 1 and not zero because always going to be greater than zero because every complex has to have query identifier
        correspondences.append({
            'HuMAP2_ID': humap2_id,
            'HuMAP3_ID': humap3_id,
            'HuMAP2_size': len(set1),
            'HuMAP3_size': len(set2),
            'shared_items': len(shared),
            'shared_elements': shared,
            'weighted_similarity': weighted_similarity(set1, set2)
        })
        used_source2_indices.add(i)

# Second pass: Fallback method for unmatched groups
# Create a list of unused source2 indices
unused_source2_indices = [
    j for j in range(len(source2_sorted)) 
    if j not in used_source2_indices
]

# Fallback matching
for humap2_id, set1 in source1_sorted:
    # Skip if this HuMAP2 ID has already been matched
    if any(corr['HuMAP2_ID'] == humap2_id for corr in correspondences):
        continue
    
    # Find best match among unused source2 groups
    best_match = None
    best_similarity = -1
    best_unused_index = -1
    
    for j in unused_source2_indices:
        humap3_id, set2 = source2_sorted[j]
        similarity = weighted_similarity(set1, set2)
        
        if similarity > best_similarity:
            best_match = (humap3_id, set2)
            best_similarity = similarity
            best_unused_index = j
    
    # Add the best match if found
    if best_match:
        humap3_id, set2 = best_match
        shared = get_shared_items(set1, set2)
        
        correspondences.append({
            'HuMAP2_ID': humap2_id,
            'HuMAP3_ID': humap3_id,
            'HuMAP2_size': len(set1),
            'HuMAP3_size': len(set2),
            'shared_items': len(shared),
            'shared_elements': shared,
            'weighted_similarity': best_similarity
        })
        
        # Remove this index from unused indices
        unused_source2_indices.remove(best_unused_index)

# Convert to DataFrame for easy viewing
df_correspondences = pd.DataFrame(correspondences)

# Sort by HuMAP2 ID size
df_correspondences = df_correspondences.sort_values('HuMAP2_size', ascending=False)

# Display results
print("\nCorrespondences:")
for _, row in df_correspondences.iterrows():
    print(f"\nHuMAP2: {row['HuMAP2_ID']} (size: {row['HuMAP2_size']})")
    print(f"HuMAP3: {row['HuMAP3_ID']} (size: {row['HuMAP3_size']})")
    print(f"Shared items: {row['shared_items']}")
    print(f"Shared elements: {row['shared_elements']}")
    print(f"Weighted similarity: {row['weighted_similarity']:.3f}")

# Optional: Save to CSV for further analysis
df_correspondences.to_csv('humap_correspondences.csv', index=False)
df_correspondences


Correspondences:

HuMAP2: HuMAP2_01834 (size: 5)
HuMAP3: huMAP3_12042.1 (size: 56)
Shared items: 4
Shared elements: {'Q8TDJ6', 'Q9GZN7', 'Q9Y4E6', 'Q9Y485'}
Weighted similarity: 0.071

HuMAP2: HuMAP2_03388 (size: 4)
HuMAP3: huMAP3_10511.1 (size: 35)
Shared items: 3
Shared elements: {'Q9Y485', 'Q9GZN7', 'Q9Y4E6'}
Weighted similarity: 0.084

HuMAP2: HuMAP2_01148 (size: 2)
HuMAP3: huMAP3_13872.1 (size: 29)
Shared items: 2
Shared elements: {'Q8TDJ6', 'Q9GZN7'}
Weighted similarity: 0.069


,HuMAP2_ID,HuMAP3_ID,HuMAP2_size,HuMAP3_size,shared_items,shared_elements,weighted_similarity
0,HuMAP2_01834,huMAP3_12042.1,5,56,4,"{Q8TDJ6, Q9GZN7, Q9Y4E6, Q9Y485}",0.070551
1,HuMAP2_03388,huMAP3_10511.1,4,35,3,"{Q9Y485, Q9GZN7, Q9Y4E6}",0.084048
2,HuMAP2_01148,huMAP3_13872.1,2,29,2,"{Q8TDJ6, Q9GZN7}",0.068966


In [28]:
# STILL TROUBLESHOOTING THIS BUT IT IS CLOSE
# make summary table (LIKE THE EARLIER ONE) with those correspondences
# Create a dictionary to map HuMAP2_ID to its corresponding details
correspondences_dict = {
    corr['HuMAP2_ID']: {
        'HuMAP3_ID': corr['HuMAP3_ID'],
        'size_df3': corr['HuMAP3_size'],
        'shared_items': corr['shared_items'],
        'weighted_similarity': corr['weighted_similarity']
    } for corr in correspondences
}

# Modify the correspondence creation to use this new matching
# First, create the base dataframe as before
df2_sorted = df2_expanded.groupby('HuMAP2_ID').size().sort_values(ascending=False)
df3_sorted = df3_expanded.groupby('HuMAP3_ID').size().sort_values(ascending=False)

# Convert the series to dataframes
df2_ranks = df2_sorted.reset_index()
df3_ranks = df3_sorted.reset_index()

# Rename columns for clarity
df2_ranks.columns = ['HuMAP2_ID', 'size_df2']
df3_ranks.columns = ['HuMAP3_ID', 'size_df3']

# Add rank columns
df2_ranks['rank'] = range(1, len(df2_ranks) + 1)
df3_ranks['rank'] = range(1, len(df3_ranks) + 1)

# Merge the rankings side by side
correspondence = pd.merge(
    df2_ranks,
    df3_ranks,
    on='rank',
    how='outer'
)
# Add details from the correspondences
def get_correspondence_details(row):
    if pd.notna(row['HuMAP2_ID']) and row['HuMAP2_ID'] in correspondences_dict:
        details = correspondences_dict[row['HuMAP2_ID']]
        return pd.Series({
            #'HuMAP3_ID': details['HuMAP3_ID'],
            #'size_df3': details['size_df3'],
            'shared_items': details['shared_items'],
            'improvement_in_v3': int(details['size_df3'] - row['size_df2'])
        })
    return pd.Series({
        #'HuMAP3_ID': '',
        #'size_df3': 0,
        'shared_items': 'na',
        'improvement_in_v3': 'na'
    })

correspondence_details = correspondence.apply(get_correspondence_details, axis=1)
correspondence = pd.concat([correspondence, correspondence_details], axis=1)
# Calculate coverage ratio
def cov_ratio(row):
    if isinstance(row['shared_items'], (int, float)) and row['size_df2'] > 0 and row['size_df3'] > 0:
        return round(row['shared_items'] / min(row['size_df2'], row['size_df3']) * 100, 2) 
    else:
        return 'na'

correspondence['coverage_ratio'] = correspondence.apply(cov_ratio, axis=1)

# Prepare display
correspondence = correspondence.sort_values('rank')
correspondence['size_df2'] = correspondence['size_df2'].fillna(0).astype(int)
correspondence['size_df3'] = correspondence['size_df3'].fillna(0).astype(int)


# Create display dataframe
display_df = correspondence.copy()
display_df['HuMAP2_ID'] = display_df['HuMAP2_ID'].fillna('')
display_df['size_df2'] = display_df['size_df2'].replace(0, '')

# Update print statement
print("\nCorrespondences (considering shared complex members) for version 2 vs. 3 with improvement, shared items, and coverage ratio:")
print(display_df[['HuMAP2_ID', 'size_df2', 'rank', 'HuMAP3_ID', 'size_df3', 'improvement_in_v3', 'shared_items', 'coverage_ratio']].to_string(index=False))


Correspondences (considering shared complex members) for version 2 vs. 3 with improvement, shared items, and coverage ratio:
   HuMAP2_ID size_df2  rank      HuMAP3_ID  size_df3 improvement_in_v3 shared_items coverage_ratio
HuMAP2_01834        5     1 huMAP3_12042.1        56                51            4           80.0
HuMAP2_03388        4     2 huMAP3_10511.1        35                31            3           75.0
HuMAP2_01148        2     3 huMAP3_13872.1        29                27            2          100.0
                          4 huMAP3_09477.1        26                na           na             na
                          5 huMAP3_09873.1        24                na           na             na
                          6 huMAP3_08678.1        22                na           na             na
                          7 huMAP3_09242.1        20                na           na             na
                          8 huMAP3_07329.1        20                na           n

In [29]:
# Make sure nothing present in HuMAP2 data disappeared in huMAP3 data
# Function to compare UniProt ACCs between HuMAP2 and HuMAP3
def check_disappearing_items(df_correspondences, df2_expanded, df3_expanded):
    # Tracking disappearing items
    disappearing_items = []
    
    for _, correspondence in df_correspondences.iterrows():
        humap2_id = correspondence['HuMAP2_ID']
        humap3_id = correspondence['HuMAP3_ID']
        
        # Get UniProt ACCs for this HuMAP2 group
        humap2_accs = set(df2_expanded[df2_expanded['HuMAP2_ID'] == humap2_id]['Uniprot_ACCs'])
        
        # Get UniProt ACCs for this HuMAP3 group
        humap3_accs = set(df3_expanded[df3_expanded['HuMAP3_ID'] == humap3_id]['Uniprot_ACCs'])
        
        # Find items in HuMAP2 that are not in HuMAP3
        lost_items = humap2_accs - humap3_accs
        
        if lost_items:
            disappearing_items.append({
                'HuMAP2_ID': humap2_id,
                'HuMAP3_ID': humap3_id,
                'lost_items': lost_items,
                'total_humap2_items': len(humap2_accs),
                'total_humap3_items': len(humap3_accs),
                'percent_lost': len(lost_items) / len(humap2_accs) * 100 if humap2_accs else 0
            })
    
    # Convert to DataFrame for easy analysis
    df_disappearing = pd.DataFrame(disappearing_items)
    
    # Display results
    global something_dropped_completely
    something_dropped_completely = False
    if not df_disappearing.empty:
        print("\nDisappearing Items between Hu.MAP2.0 and Hu.MAP3.0 data:")
        for _, row in df_disappearing.iterrows():
            print(f"\nHuMAP2 ID: {row['HuMAP2_ID']} -> HuMAP3 ID: {row['HuMAP3_ID']}")
            print(f"Total items in HuMAP2: {row['total_humap2_items']}")
            print(f"Total items in HuMAP3: {row['total_humap3_items']}")
            print(f"Percent of items lost: {row['percent_lost']:.2f}%")
            print("UniProt ACCs Lost From Corresponding Complex:")
            for item in row['lost_items']:
                dropped_completely = (item not in df3_expanded['Uniprot_ACCs'].to_list())
                if dropped_completely:
                    print(f"  - {item} (AND NOT IN ANY OTHER Hu.MAP3.0 {search_term}-complexes!!!)")
                    something_dropped_completely = True
                else:
                    print(f"  - {item} (But present in other, {search_term}-related Hu.MAP3.0 complexes)")
        
        # Optional: Save to CSV
        df_disappearing.to_csv('disappearing_items.csv', index=False)
        
        return df_disappearing, something_dropped_completely
    else:
        print("\nNo disappearing items found between HuMAP2 and HuMAP3.")
        return None, something_dropped_completely

# Call the function
disappearing_df, something_dropped_completely = check_disappearing_items(df_correspondences, df2_expanded, df3_expanded)


Disappearing Items between HuMAP2 and HuMAP3:

HuMAP2 ID: HuMAP2_01834 -> HuMAP3 ID: huMAP3_12042.1
Total items in HuMAP2: 5
Total items in HuMAP3: 56
Percent of items lost: 20.00%
UniProt ACCs Lost From Corresponding Complex:
  - P50993 (AND NOT IN ANY OTHER Hu.MAP3.0 ROGDI-complexes!!!)

HuMAP2 ID: HuMAP2_03388 -> HuMAP3 ID: huMAP3_10511.1
Total items in HuMAP2: 4
Total items in HuMAP3: 35
Percent of items lost: 25.00%
UniProt ACCs Lost From Corresponding Complex:
  - Q8TDJ6 (But present in other, ROGDI-related Hu.MAP3.0 complexes)


In [47]:
# TO DO: Also make sure nothing that is in the majority of complexes in hu.MAP 2.0 data disappears entirely from hu.MAP 3.0 data.
# That can only occur as the case if `something_dropped_completely`, that we've already check on, is True
majority_item_dropped_completely = False #set up by declaring false
if something_dropped_completely:
    # collect information on what identifiers occur in the majority of complexes in hu.MAP 2.0 data
    # the search term should always be present and can thus serve as a sanity check for this collection
    majority_identifiers = []
    def find_majority_uniprot_accs(df):
        # Group by HuMAP2_ID and get unique Uniprot_ACCs for each complex
        complex_accs = df.groupby('HuMAP2_ID')['Uniprot_ACCs'].unique()
        
        # Count total number of unique complexes
        total_complexes = len(complex_accs)
        
        # Flatten and count occurrences of each Uniprot ACC across complexes
        all_accs = df['Uniprot_ACCs'].tolist()
        acc_counts = pd.Series(all_accs).value_counts()
        
        # Find ACCs that appear in more than 50% of complexes
        majority_accs = acc_counts[acc_counts >= 0.51 * total_complexes].index.tolist()
        
        return majority_accs
    if find_majority_uniprot_accs(df2_expanded):
        majority_identifiers = find_majority_uniprot_accs(df2_expanded)
    # Now check none of those `majority_identifiers` match the disappearing ones dropped completely
    print("\nHu.MAP2.0-Majority Complex Members Disappearing in Hu.MAP3.0 data:")
    for item in list(set.union(*disappearing_df['lost_items'])): # note `list(set.union(*disappearing_df['lost_items']))` gets the unique set members from the 'lost_items' column
        if item in majority_identifiers:
            majority_item_dropped_completely = (item not in df3_expanded['Uniprot_ACCs'].to_list())
            if majority_item_dropped_completely:
                print(f"  - {item} IS IN MAJORITY OF Hu.MAP2.0 COMPLEXES YET NOT IN ANY Hu.MAP3.0 {search_term}-complexes!!! (PERHAPS CONCERNING?)")
                majority_item_dropped_completely = True
    if majority_item_dropped_completely == False:
        print("NONE")


Hu.MAP2.0-Majority Complex Members Disappearing in Hu.MAP3.0 data:
NONE


In [21]:
# PROBABLY DON'T NEED THIS BUT RECORDING IN CASE LATER I DO
# Show IDs that don't have correspondences
print("\nHuMAP2_IDs without correspondences:")
print(correspondence[correspondence['HuMAP3_ID'].isna()][['HuMAP2_ID', 'size_df2']])

print("\nHuMAP3_IDs without correspondences:")
print(correspondence[correspondence['HuMAP2_ID'].isna()][['HuMAP3_ID', 'size_df3']])


HuMAP2_IDs without correspondences:
Empty DataFrame
Columns: [HuMAP2_ID, size_df2]
Index: []

HuMAP3_IDs without correspondences:
         HuMAP3_ID  size_df3
3   huMAP3_09477.1        26
4   huMAP3_09873.1        24
5   huMAP3_08678.1        22
6   huMAP3_09242.1        20
7   huMAP3_07329.1        20
8   huMAP3_08381.1        20
9   huMAP3_07099.1        18
10  huMAP3_01501.1        15
11  huMAP3_01899.1        11
12  huMAP3_03469.1         4
13  huMAP3_05952.1         2


In [ ]:
import time

def executeSomething():
    #code here
    print ('.')
    time.sleep(480) #60 seconds times 8 minutes

while True:
    executeSomething()

.
.
.
.


 You can change the hard-coded values here to investigate the differences for others. However, chances are you are interested in the differnces for several genes/proteins, and so you'll want to first check out the next notebook in this series, ['Using snakemake to highlight differences between hu.MAP 2.0 and hu.MAP 3.0 data for multiple identifiers'](Using_snakemake_to_highlight_differences_between_hu.MAP2_and_hu.MAP3_data_for_multiple_indentifiers.ipynb). That notebook will work through the steps to run a Snakemake workflow to process the identifiers for many proteins/genes to generate summary reports highlighting differences in human complexes reported in hu.MAP 2.0 vs. hu.MAP 3.0 data.
 
-----

Enjoy!

See my [humap3-binder repo](https://github.com/fomightez/humap3-binder) and [humap3-utilities](https://github.com/fomightez/structurework/humap3-utilities) for related information & resources for this notebook.



-----
